In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os

MODEL_NAME = 'Centernet_MobileNet_V2_FPN_512x512'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz'
TFRECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [2]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',MODEL_NAME, 'export'), 
#     'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
#     'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
}

In [3]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TFRECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [4]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# Install tensorflow object-detection API

In [5]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

Cloning into 'Tensorflow/models'...
remote: Enumerating objects: 57779, done.
remote: Counting objects: 100% (201/201), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 57779 (delta 89), reused 119 (delta 15), pack-reused 57578
Receiving objects: 100% (57779/57779), 572.97 MiB | 30.69 MiB/s, done.
Resolving deltas: 100% (40032/40032), done.


In [6]:
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e .

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Processing /content/Tensorflow/models/research
     |████████████████████████████████| 9.6MB 7.5MB/s 
     |████████████████████████████████| 358kB 45.8MB/s 
     |████████████████████████████████| 1.6MB 44.3MB/s 
     |████████████████████████████████| 153kB 44.7MB/s 
     |████████████████████████████████| 61kB 8.7MB/s 
     |████████████████████████████████| 2.3MB 42.1MB/s 
     |████████████████████████████████| 829kB 45.3MB/s 
     |████████████████████████████████| 645kB 37.8MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 686kB 37.6MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
     |████████████████████████████████| 1.2MB 44.1MB/s 
     |████████████████████████████████| 38.2MB 78kB/s 
   

In [7]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

2021-06-04 05:39:23.372936: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-06-04 05:39:25.501741: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-04 05:39:25.563749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-06-04 05:39:25.564405: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-06-04 05:39:25.564448: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfull

In [8]:
!git clone https://github.com/hai-h-nguyen/Yolo2Pascal-annotation-conversion.git {paths['SCRIPTS_PATH']}
!rm -r /content/Tensorflow/scripts/demo /content/Tensorflow/scripts/pascal2yolo

Cloning into 'Tensorflow/scripts'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 122 (delta 48), reused 46 (delta 18), pack-reused 25
Receiving objects: 100% (122/122), 285.29 KiB | 11.41 MiB/s, done.
Resolving deltas: 100% (54/54), done.


# Handle Images

In [ ]:
!wget https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip
!unzip PennFudanPed.zip -d {paths['IMAGE_PATH']}

In [ ]:
!pip install pyqt5

     |████████████████████████████████| 8.3MB 9.3MB/s 
     |████████████████████████████████| 59.9MB 44kB/s 
     |████████████████████████████████| 317kB 36.2MB/s 


In [ ]:
!python {paths['SCRIPTS_PATH']}/yolo2pascal/yolo2voc.py \
  {paths['IMAGE_PATH']}/PennFudanPed/Annotation

Convert PennPed00072.txt
Load this image: Tensorflow/workspace/images/PennFudanPed/Annotation/PennPed00072.jpg
Traceback (most recent call last):
  File "Tensorflow/scripts/yolo2pascal/yolo2voc.py", line 37, in <module>
    tYoloParseReader = YoloReader(txtPath, image)
  File "/content/Tensorflow/scripts/yolo2pascal/yolo_io.py", line 112, in __init__
    self.parseYoloFormat()
  File "/content/Tensorflow/scripts/yolo2pascal/yolo_io.py", line 146, in parseYoloFormat
    classIndex, xcen, ycen, w, h = bndBox.split(' ')
ValueError: too many values to unpack (expected 5)
